In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import time
import re

In [2]:
directory = '/projectnb/oleds/stannir4/ADAPT'
exception = ['CREST','VG']
computation_cost = pd.DataFrame(columns=['molecule','conformation','image','core','day','hour','minute','second','milli-second'])
error_list = pd.DataFrame(columns=['molecule','conformation','image','error'])

In [3]:
t = time.time()
for molecule in os.listdir(directory):   #for each cruciform
    #print(molecule)
    molecule_path = os.path.join(directory, molecule)
    
    for conformation in os.listdir(molecule_path):   #for each conformation
        #print(conformation)
        
        if conformation not in exception:
            conformation_path = os.path.join(molecule_path, conformation,'GROUND')
            inp_path = os.path.join(conformation_path,'ground.inp')
            out_path = os.path.join(conformation_path,'ground.out')
            
            if Path(inp_path).is_file() == True:
                with open(inp_path,'rt') as inp:
                    inp_line = inp.readlines()[2]
                    cores = inp_line.split()[2]   #number of cores
            else:
                cores = '-'
                error_list.loc[len(error_list)] = [molecule,conformation,'-','inp not found']
            
            if Path(out_path).is_file() == True:
                with open(out_path,'rt') as out:
                    out_line = out.readlines()[-1]
                    if out_line == '\n':
                        run_time = [np.nan]*5   #aborting the run
                        error_list.loc[len(error_list)] = [molecule,conformation,'-','aborting the run']
                    else:
                        run_time = out_line.split()[3:13:2]   #running time
            else:
                run_time = [np.nan] * 5
                error_list.loc[len(error_list)] = [molecule,conformation,'-','out not found']
            
            computation_cost.loc[len(computation_cost)] = [molecule,conformation,'-',cores]+run_time
            
            for image in os.listdir(conformation_path):
                image_path = os.path.join(conformation_path,image)
                if os.path.isdir(image_path):
                    sub_inp_path = os.path.join(image_path,'ground.inp')
                    sub_out_path = os.path.join(image_path,'ground.out')
                    
                    if Path(sub_inp_path).is_file() == True:
                        with open(sub_inp_path,'rt') as inp:
                            inp_line = inp.readlines()[2]
                            cores = inp_line.split()[2]   #number of cores
                    else:
                        cores = '-'
                        error_list.loc[len(error_list)] = [molecule,conformation,image,'inp not found']
                    if Path(sub_out_path).is_file() == True:
                        with open(sub_out_path,'rt') as out:
                            out_line = out.readlines()[-1]
                            if out_line == '\n':
                                run_time = [np.nan]*5   #aborting the run
                                error_list.loc[len(error_list)] = [molecule,conformation,image,'aborting the run']
                            else:
                                run_time = out_line.split()[3:13:2]   #running time
                    else:
                        run_time = [np.nan] * 5
                        error_list.loc[len(error_list)] = [molecule,conformation,image,'out not found']
                    
                    computation_cost.loc[len(computation_cost)] = [molecule,conformation,image,cores]+run_time
        
        elif conformation == 'CREST':
            conformation_path = os.path.join(molecule_path, conformation)
            out_path = os.path.join(conformation_path,(molecule+'.out'))
            if Path(out_path).is_file() == True:
                with open(out_path,'rt') as out:
                    out_line = out.readlines()[-3]
                    hour, minute, second = re.findall(r'\d+', out_line)
                    run_time = [0,hour,minute,second,0]
            else:
                run_time = [np.nan] * 5
                error_list.loc[len(error_list)] = [molecule,conformation,'-','out not found']
            computation_cost.loc[len(computation_cost)] = [molecule,conformation,'-',cores]+run_time
            
        else:
            conformation_path = os.path.join(molecule_path, conformation)
            out_path = os.path.join(conformation_path,'vg.out')
            inp_path = os.path.join(conformation_path,'vg.inp')
            
            if Path(inp_path).is_file() == True:
                with open(inp_path,'rt') as inp:
                    inp_line = inp.readlines()[3]
                    cores = inp_line.split()[2]   #number of cores
            else:
                cores = '-'
                error_list.loc[len(error_list)] = [molecule,conformation,'-','inp not found']
            
            if Path(out_path).is_file() == True:
                with open(out_path,'rt') as out:
                    out_line = out.readlines()[-1]
                    if out_line == '\n':
                        run_time = [np.nan]*5   #aborting the run
                        error_list.loc[len(error_list)] = [molecule,conformation,'-','aborting the run']
                    else:
                        run_time = out_line.split()[3:13:2]   #running time
            else:
                run_time = [np.nan] * 5
                error_list.loc[len(error_list)] = [molecule,conformation,'-','out not found']
print(time.time()-t)

217.2650649547577


In [4]:
computation_cost

,molecule,conformation,image,core,day,hour,minute,second,milli-second
0,M_pr,6,-,16,0,0,13,2,935
1,M_pr,3,-,16,0,0,26,46,70
2,M_pr,3,P30,16,0,0,21,54,904
3,M_pr,3,M10,16,0,0,32,16,793
4,M_pr,3,M40,16,0,0,33,57,37
...,...,...,...,...,...,...,...,...,...
13902,U_pr,CREST,-,16,0,0,30,41,0
13903,U_pr,9,-,16,0,1,53,23,213
13904,U_pr,12,-,16,0,0,55,52,386
13905,U_pr,10,-,16,0,1,36,32,666


In [5]:
error_list

,molecule,conformation,image,error
0,M_pr,3,P10,inp not found
1,M_pr,3,P10,out not found
2,A7_pr,6,P10,inp not found
3,A7_pr,6,P10,out not found
4,A7_pr,5,P5,inp not found
...,...,...,...,...
5702,X_pr,2,P25,out not found
5703,X_pr,4,P15,inp not found
5704,X_pr,4,P15,out not found
5705,X_pr,1,P5,inp not found


In [6]:
df = computation_cost.sort_values(by=['molecule','conformation']).dropna()
df.to_csv('/projectnb/oleds/Yuning Pan/computation_cost(Shama).csv', index=False)

In [7]:
df = pd.read_csv (r'/projectnb/oleds/Yuning Pan/computation_cost(Shama).csv')

In [8]:
from datetime import timedelta

In [9]:
cost_per_mol = pd.DataFrame(columns = ['molecule','days','hours','minutes','seconds','milli-seconds'])
name = df['molecule'][0]
sum_d, sum_h, sum_m, sum_s, sum_ms = 0, 0, 0, 0, 0
for i in range(len(df)):
    if df.iloc[i,0] != name or i == len(df)-1:
        cost_per_mol.loc[len(cost_per_mol)] = [name, sum_d, sum_h, sum_m, sum_s, sum_ms]
        name = df.iloc[i,0]
        sum_d, sum_h, sum_m, sum_s, sum_ms = df.iloc[i,4], df.iloc[i,5], df.iloc[i,6], df.iloc[i,7], df.iloc[i,8]
    else:
        sum_d += df.iloc[i,4]
        sum_h += df.iloc[i,5]
        sum_m += df.iloc[i,6]
        sum_s += df.iloc[i,7]
        sum_ms += df.iloc[i,8]

In [10]:
second, cost_per_mol['milli-seconds'] = cost_per_mol['milli-seconds']//1000, cost_per_mol['milli-seconds']%1000
minute, cost_per_mol['seconds'] = (cost_per_mol['seconds']+second)//60, (cost_per_mol['seconds']+second)%60
hour, cost_per_mol['minutes'] = (cost_per_mol['minutes']+minute)//60, (cost_per_mol['minutes']+minute)%60
cost_per_mol['hours'] += hour

In [11]:
hours = (cost_per_mol['minutes']*60 + cost_per_mol['seconds'])/3600
df1 = pd.DataFrame(columns=['molecule','hours'])
df1['molecule'] = cost_per_mol['molecule']
df1['hours'] = cost_per_mol['hours'] + hours

In [12]:
df1.to_csv('/projectnb/oleds/Yuning Pan/cost_per_mol(Shama).csv', index=False)

In [13]:
error_list.to_csv('/projectnb/oleds/Yuning Pan/error_list(Shama).csv', index=False)

In [14]:
error_list.shape

(5707, 4)

In [15]:
df[df['conformation']=='CREST']

,molecule,conformation,image,core,day,hour,minute,second,milli-second
13,A1,CREST,-,16,0,1,7,59,0
15,A10,CREST,-,16,0,0,16,18,0
18,A10P,CREST,-,16,0,0,9,26,0
40,A10_pr,CREST,-,16,0,0,8,53,0
95,A11,CREST,-,16,0,0,30,29,0
...,...,...,...,...,...,...,...,...,...
10876,VT26P48C2,CREST,-,16,0,0,40,6,0
10916,VT26P48C3,CREST,-,16,0,0,37,45,0
10939,V_pr,CREST,-,16,0,0,10,59,0
10960,W_pr,CREST,-,16,0,0,11,23,0


In [5]:
df = pd.read_csv (r'/projectnb/oleds/Yuning Pan/cost_per_mol(Shama).csv')

In [6]:
df.head()

,molecule,hours
0,A1,48.127778
1,A10,1.451111
2,A10P,0.922778
3,A10_pr,5.132778
4,A11,92.162222


In [9]:
np.where(df['molecule'] == 'P_pr')

(array([85]),)